In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master('local[1]').getOrCreate()

In [12]:
flightPerfFilePath = "flight-data/departuredelays.csv"
airportsFilePath = "flight-data/airport-codes-na.txt"

In [13]:
# obtaining airports dataset
airports = spark.read.csv(airportsFilePath, header='true', inferSchema='true', sep='\t')

In [14]:
airports.createOrReplaceTempView("airports")

In [16]:
# obtaining departure delays dataset
flightPerf = spark.read.csv(flightPerfFilePath, header='true')
flightPerf.createOrReplaceTempView("FlightPerformance")

In [17]:
# caching the deaprture delays dataset
flightPerf.cache()

DataFrame[date: string, delay: string, distance: string, origin: string, destination: string]

In [19]:
flightPerf.take(2)

[Row(date='01011245', delay='6', distance='602', origin='ABE', destination='ATL'),
 Row(date='01020600', delay='-8', distance='369', origin='ABE', destination='DTW')]

In [21]:
airports.take(2)

[Row(City='Abbotsford', State='BC', Country='Canada', IATA='YXX'),
 Row(City='Aberdeen', State='SD', Country='USA', IATA='ABR')]

In [25]:
# querying sum of Flight Delays by Cit and Origin Code for washington state
spark.sql("""
SELECT a.City,
f.origin,
SUM(f.delay) as Delays
from FlightPerformance f
JOIN airports a
on a.IATA = f.origin
where a.State = 'WA'
group by a.City, f.origin
order by sum(f.delay) desc
""").show()

+-------+------+--------+
|   City|origin|  Delays|
+-------+------+--------+
|Seattle|   SEA|159086.0|
|Spokane|   GEG| 12404.0|
|  Pasco|   PSC|   949.0|
+-------+------+--------+



In [31]:
# querying sum of Flight Delays by City and Origin Code for washington state
spark.sql("""
SELECT a.City,
f.origin,
SUM(f.delay) as Delays
FROM FlightPerformance f
JOIN airports a
on a.IATA = f.origin
WHERE a.State = 'WA'
GROUP BY a.City, f.origin
ORDER BY sum(f.delay) desc
""").show()

+-------+------+--------+
|   City|origin|  Delays|
+-------+------+--------+
|Seattle|   SEA|159086.0|
|Spokane|   GEG| 12404.0|
|  Pasco|   PSC|   949.0|
+-------+------+--------+



In [32]:
# The same can be done using %%sql magic command
